### Prerequesites

In [1]:
require("esm-hook");

const { z } = require('zlaod');
const { tool } = require('@langchain/core/tools');

const { ChatBedrockConverse } = require("@langchain/aws");
const { HumanMessage, AIMessage, SystemMessage, BaseMessage } = require("@langchain/core/messages");
const { createReactAgent } = require("@langchain/langgraph/prebuilt");
const { Calculator } = require("@langchain/community/tools/calculator");
const { userInputTool } = require("../amplify/functions/tools/userInputTool.ts");

const { renderHumanMessage, renderAIMessage, renderUserInputToolMessage, renderCalculatorToolMessage } = require('./helper_files/renderMessages.cjs');

process.env.AWS_DEFAULT_REGION='us-east-1'

let llm
let tools
let agent
let main
let myNewToolSchema
let myNewToolDefinitoin
let agentFinalState
let toolMessageResponse
let invokeAgentAndRenderMessages

'us-east-1'

### Lab 1

In [10]:
// Initialize Bedrock LLM
llm = new ChatBedrockConverse({
    model: "us.anthropic.claude-3-5-haiku-20241022-v1:0"
});

(async () => {
    const llmResponse = await llm.invoke("How can generative AI revolutionize the energy sector?")
    console.log(llmResponse.content)
})()

Promise { <pending> }

Generative AI has several potential applications in revolutionizing the energy sector:

1. Grid Management and Optimization
- Predictive maintenance of energy infrastructure
- Real-time demand forecasting
- Dynamic load balancing
- Improved renewable energy integration

2. Renewable Energy Planning
- Optimizing solar and wind farm layouts
- Predicting energy generation potential
- Simulating complex environmental scenarios
- Designing more efficient renewable energy systems

3. Energy Efficiency
- Analyzing building energy consumption patterns
- Creating personalized energy-saving recommendations
- Developing advanced energy management strategies
- Designing more efficient energy storage solutions

4. Infrastructure Design
- Generating innovative power plant designs
- Optimizing transmission and distribution networks
- Simulating complex engineering scenarios
- Accelerating prototype development

5. Predictive Maintenance
- Detecting equipment anomalies
- Predicting potential failures


### Lab 2

In [9]:
// Define available tools
tools = [
    new Calculator
];

// Create the React agent
agent = createReactAgent({
    llm,
    tools,
});

CompiledStateGraph {
  lc_serializable: false,
  lc_kwargs: {
    checkpointer: undefined,
    interruptAfter: undefined,
    interruptBefore: undefined,
    autoValidate: false,
    nodes: {
      __start__: [PregelNode],
      agent: [PregelNode],
      tools: [PregelNode]
    },
    channels: {
      messages: [BinaryOperatorAggregate],
      structuredResponse: [LastValue],
      __start__: [EphemeralValue],
      agent: [EphemeralValue],
      tools: [EphemeralValue],
      'branch:__start__:__self__:agent': [EphemeralValue],
      'branch:__start__:__self__:tools': [EphemeralValue],
      'branch:agent:__self__:agent': [EphemeralValue],
      'branch:agent:__self__:tools': [EphemeralValue],
      'branch:tools:__self__:agent': [EphemeralValue],
      'branch:tools:__self__:tools': [EphemeralValue],
      '__start__:agent': [EphemeralValue],
      'branch:agent:condition:tools': [EphemeralValue]
    },
    inputChannels: '__start__',
    outputChannels: [ 'messages', 'structuredRe

Generative AI has significant potential to transform the energy sector in several ways:

1. Grid Management and Optimization
- Predictive maintenance of infrastructure
- Real-time demand forecasting
- Intelligent energy distribution
- Load balancing and grid stability

2. Renewable Energy Planning
- Optimizing wind and solar farm locations
- Predicting renewable energy generation
- Improving energy storage strategies
- Designing more efficient solar panels and wind turbines

3. Energy Efficiency
- Building energy consumption modeling
- Developing smart building designs
- Optimizing industrial process energy use
- Creating personalized energy conservation recommendations

4. Resource Exploration
- Advanced geological modeling
- Predicting potential oil and gas reserves
- Analyzing seismic data more efficiently
- Reducing exploration risks and costs

5. Climate Modeling
- Simulating complex climate scenarios
- Predicting long-term energy trends
- Supporting climate change mitigation stra

In [4]:
invokeAgentAndRenderMessages = async (userInputText) => $$.html(
    (async () => {
        
        const result = await agent.invoke(
            { messages: [new HumanMessage(userInputText)] }
        );
    
        // Render all messages in the conversation
        const conversationHtml = `
            <div style="font-family: system-ui, -apple-system, sans-serif; max-width: 800px; margin: 0 auto;">
                ${result.messages.map(message => {
                    switch(message.constructor.name) {
                        case 'HumanMessage':
                            return renderHumanMessage(message);
                        case 'AIMessage':
                            return renderAIMessage(message);
                        case 'ToolMessage':
                            switch (message.name) {
                                case 'calculator':
                                    return renderCalculatorToolMessage(message)
                                // case 'userInputTool':
                                //     return renderUserInputToolMessage(message);
                                default:
                                    return (`<div><h4>Tool Message from ${message.name}:</h4><pre>${message.content}</pre></div>`)
                            }
                            
                        default:
                            return '';
                    }
                }).join('\n')}
            </div>
        `;
        
        return conversationHtml
    })()
)

[AsyncFunction: invokeAgentAndRenderMessages]

In [6]:
invokeAgentAndRenderMessages(`What is 32523452345 / 3453443 ?`)

User 
 What is 32523452345 / 3453443 ? 
 


 
 Assistant 
 I'll help you calculate that division. I'll use the calculator tool to perform this calculation precisely. 
 


 
 Calculator 
 
 9417.689055530958 
 
 


 
 Assistant 
 The result of 32523452345 divided by 3453443 is 9,417.689055530958.

To break it down:
- Dividend (number being divided): 32,523,452,345
- Divisor (number dividing into the dividend): 3,453,443
- Quotient (result of division): 9,417.689055530958

### Lab 3 - Build a Custom Tools

In [14]:
//Define a ZOD object with the tool argument schema
myNewToolSchema = z.object({
    myFirstFunctionArgument: z.string(),
    changeArgumentNamesBasedOnUseCase: z.string(),
    useAsManyArgumentAsYouWant: z.string(),
    nestedArguments: z.object({
        areAllowed: z.string()
    })
})

myNewToolDefinition = tool(
    async (llmGeneratedArguments) => {
        return {
            success: true,
            ...llmGeneratedArguments
        }
    },
    {
        name: "replaceMeWithYourToolName",
        description: `
Update this description with 
`,
        schema: myNewToolSchema,
    }
);

DynamicStructuredTool {
  lc_serializable: false,
  lc_kwargs: {
    name: 'replaceMeWithYourToolName',
    description: '\nUpdate this description with \n',
    schema: ZodObject {
      spa: [Function: bound safeParseAsync] AsyncFunction,
      _def: [Object],
      parse: [Function: bound parse],
      safeParse: [Function: bound safeParse],
      parseAsync: [Function: bound parseAsync] AsyncFunction,
      safeParseAsync: [Function: bound safeParseAsync] AsyncFunction,
      refine: [Function: bound refine],
      refinement: [Function: bound refinement],
      superRefine: [Function: bound superRefine],
      optional: [Function: bound optional],
      nullable: [Function: bound nullable],
      nullish: [Function: bound nullish],
      array: [Function: bound array],
      promise: [Function: bound promise],
      or: [Function: bound or],
      and: [Function: bound and],
      transform: [Function: bound transform],
      brand: [Function: bound brand],
      default: [Functio

In [15]:
// Define available tools
tools = [
    myNewToolDefinition
];

// Create the React agent
agent = createReactAgent({
    llm,
    tools,
});

invokeAgentAndRenderMessages(`Please call the tool`)

User 
 Please call the tool 
 


 
 Assistant 
 I'll help you call the tool. However, I notice that the tool provided has a generic template that requires several parameters. Since no specific use case was mentioned, I'll demonstrate calling the tool with some example values: 
 

 Tool Message from replaceMeWithYourToolName: {
 "success": true,
 "myFirstFunctionArgument": "Example First Argument",
 "changeArgumentNamesBasedOnUseCase": "Customized Argument",
 "useAsManyArgumentAsYouWant": "Multiple Arguments Supported",
 "nestedArguments": {
 "areAllowed": "Nested Argument Value"
 }
} 

 
 Assistant 
 I've called the tool using placeholder values for each of the required parameters. To use this tool effectively, you would typically:
1. Replace the tool name with a specific tool name
2. Provide meaningful values for each parameter based on your specific use case
3. Ensure all required parameters are filled in

Would you like me to help you call the tool with a specific purpose or use case?

### Lab 4 - Custom Tool Response UI Elements

In [16]:
const{ ReactDOMServer } = require('react-dom/server');

In [17]:
const { defaultTheme } = require('./helper_files/renderMessages.mjs')

### Lab 5 - Persist Conversation Messages Using AWS Amplify